# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
from unityagents import UnityEnvironment
import numpy as np

#JRF - New imports for the management of NN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#JRF - New imports for the management of Agent
import random
import copy
from collections import namedtuple, deque
from datetime import datetime

#JRF - For plotting results
import matplotlib.pyplot as plt
%matplotlib inline

BUFFER_SIZE = int(1e8)  # replay buffer size
#BATCH_SIZE = 128        # minibatch size
BATCH_SIZE = 1024        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
#LR_ACTOR = 1e-4         # learning rate of the actor 
#LR_CRITIC = 1e-3        # learning rate of the critic
#WEIGHT_DECAY = 0        # L2 weight decay
EPS_INITIAL = 1.0       # Initial Noise.
EPS_DECAY = 0.9995       # Noise decay
EPS_END = 0.01          # Min Noise
UPDATE_FREQ = 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [3]:
env = UnityEnvironment(file_name='.\Tennis_Windows_x86_64\Tennis.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print("Brain name:",brain_name)

Brain name: TennisBrain


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Create Neural Nets

Create the Neural Netwok models

In [6]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

# neural net structure based on this thread: https://knowledge.udacity.com/questions/277763

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=512, fc2_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.batch_norm1 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.leaky_relu(self.batch_norm1(self.fc1(state)))
        x = F.leaky_relu(self.fc2(x))
        return F.tanh(self.fc3(x))

class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=512, fc2_units=256): #,fc3_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.batch_norm1 = nn.BatchNorm1d(fc2_units)
        self.fc3 = nn.Linear(fc2_units,1)

#        self.fc3 = nn.Linear(fc2_units,fc3_units)
#        self.fc4 = nn.Linear(fc3_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
        #self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
        #self.fc4.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.leaky_relu(self.fcs1(state))
        #xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.leaky_relu(self.batch_norm1(self.fc2(x)))
        #x = F.relu(self.batch_norm1(self.fc2(x)))
        #x = F.leaky_relu(self.fc3(x))
        #return self.fc4(x)
        return self.fc3(x)
    


### 4. Create the agent


In [7]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size, random_seed, num_agents):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
            num_agents (int): number of agents
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)
        self.eps = EPS_INITIAL

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
#         self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters()) # default ADAM params

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
#         self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters()) # default ADAM params

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    def step(self, states, actions, rewards, next_states, dones, update):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(states, actions, rewards, next_states, dones)

        # Learn, if enough samples are available in memory
        if update & (len(self.memory) > BATCH_SIZE):
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def act(self, states, add_noise=True):
        """Returns actions for given state as per current policy."""
        states = torch.from_numpy(states).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            actions = self.actor_local(states).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            for action in actions:
                action += self.eps * self.noise.sample()
            self.eps = max(EPS_DECAY * self.eps, EPS_END)
        return np.clip(actions, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
#         dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        dx = self.theta * (self.mu - x) + self.sigma * np.array([np.random.randn() for i in range(len(x))]) # normally dist noise
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, states, actions, rewards, next_states, dones):
        """Add a new experience to memory."""
        for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
            e = self.experience(state, action, reward, next_state, done)
            self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 5a. Agent training in the environment

In [8]:
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2, num_agents=num_agents)
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
agent.reset()

#Load previously trained agent: 
#agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
#agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))


def ddpg(n_episodes=5000, max_t=2000, print_every=50):
    scores_deque = deque(maxlen=100)
    scores_mean = []
    max_score = -np.Inf
    avgscore=0
    maxscore=0
    minscore=1
    
    for i_episode in range(1, n_episodes+1):
        #print("i_episode:",i_episode)
        env_info = env.reset(train_mode=True)[brain_name]  
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents) 
        agent.reset()
        now = datetime.now()
        
        for t in range(max_t):
            #print("t:",t)
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent) 
            #print("scores:",scores)
            if t % UPDATE_FREQ == 0:
                update = True
            agent.step(states, actions, rewards, next_states, dones, update)
            update = False
            states = next_states 
            if np.any(dones):
                break 
        #print("scores:",scores)

        #scores_deque.append(np.max(scores,initial=0))
        #scores_mean.append(scores_deque)
        scores_deque.append(np.mean(scores))
        scores_mean.append(np.mean(scores))
        
        avg_score = np.mean(scores_deque)

        
        if avg_score>maxscore: maxscore=float(avg_score)
        if avg_score<minscore: minscore=float(avg_score)

        timenow= "{:02}".format(now.hour) + ":"+ "{:02}".format(now.minute) + ":" + "{:02}".format(now.second)
        #print("scores:",scores)
        #print("scores_mean",scores_mean)
        print('\rEpisode {}\tAverage Score: {:.2f} \tMinScore: {:.2f} \tMaxScore: {:.2f} \tHour: {}'.format(i_episode, avg_score,minscore, maxscore, timenow), end="")
        #print('Episode {}\tAverage Score: {:.2f} \tMinScore: {:.2f} \tMaxScore: {:.2f} \tHour: {}'.format(i_episode, avg_score,minscore, maxscore, timenow))
        #if avg_score > 30 and avgscore <= score: # only save when average is better and game have been passed. 
        if i_episode % 20 == 0: 
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f} \tMinScore: {:.2f} \tMaxScore: {:.2f} \tHour: {}'.format(i_episode, avg_score,minscore, maxscore, timenow))
        if i_episode % 100 == 0: 
            print("Reset values avgscore, max and minscore")
            avgscore=0
            maxscore=0
            minscore=1
    
    return scores_mean

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

C:\Users\javro\.conda\envs\drlnd\lib\site-packages\torch\nn\functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 50	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:04:40
Episode 100	Average Score: 0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:06:39
Reset values avgscore, max and minscore
Episode 150	Average Score: 0.00 	MinScore: 0.00 	MaxScore: 0.00 	Hour: 13:08:04
Episode 200	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:09:26
Reset values avgscore, max and minscore
Episode 250	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:10:50
Episode 300	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:12:12
Reset values avgscore, max and minscore
Episode 350	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:13:38
Episode 400	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:15:12
Reset values avgscore, max and minscore
Episode 450	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:16:31
Episode 500	Average Score: -0.00 	MinScore: -0.00 	MaxScore: 0.00 	Hour: 13:17:54
Reset values avgscore, m

Episode 4050	Average Score: 0.39 	MinScore: 0.39 	MaxScore: 0.46 	Hour: 22:20:03
Episode 4100	Average Score: 0.05 	MinScore: 0.05 	MaxScore: 0.46 	Hour: 22:23:19
Reset values avgscore, max and minscore
Episode 4150	Average Score: 0.05 	MinScore: 0.05 	MaxScore: 0.05 	Hour: 22:27:18
Episode 4200	Average Score: 0.07 	MinScore: 0.05 	MaxScore: 0.07 	Hour: 22:32:03
Reset values avgscore, max and minscore
Episode 4250	Average Score: 0.11 	MinScore: 0.07 	MaxScore: 0.11 	Hour: 22:39:27
Episode 4300	Average Score: 0.14 	MinScore: 0.07 	MaxScore: 0.14 	Hour: 22:45:22
Reset values avgscore, max and minscore
Episode 4350	Average Score: 0.08 	MinScore: 0.08 	MaxScore: 0.13 	Hour: 22:49:01
Episode 4400	Average Score: 0.03 	MinScore: 0.03 	MaxScore: 0.13 	Hour: 22:51:38
Reset values avgscore, max and minscore
Episode 4450	Average Score: 0.02 	MinScore: 0.02 	MaxScore: 0.03 	Hour: 22:53:58
Episode 4500	Average Score: 0.02 	MinScore: 0.02 	MaxScore: 0.03 	Hour: 22:56:46
Reset values avgscore, max and

KeyboardInterrupt: 

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
print("len(scores):",len(scores))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1901, 2201), scores[1900:2200])
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

#plt.plot(np.arange(1, 201), scores[000:200])
#plt.ylabel('Score')
#plt.xlabel('Episode #')
#plt.show()

NameError: name 'scores_mean' is not defined

### 6a. Trained agent interacting in the environment

In [12]:
# Watch the trained agent
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2, num_agents=num_agents)

agent.reset()

#Load previously trained agent: 
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations            # get the current state

ep_scores = np.zeros(num_agents)                                          # initialize the score

for i in range(1000):
    while True:
        actions = agent.act(states)        # select an action from the trained agent
        env_info = env.step(actions)[brain_name]        # send the action to the environment
        next_states = env_info.vector_observations   # get the next state
        rewards = env_info.rewards                   # get the reward
        dones = env_info.local_done                  # see if episode has finished
        if np.array(rewards)[0]> 0: 
            ep_scores[0]+= np.array(rewards)[0]
        if np.array(rewards)[1]> 0: 
            ep_scores[1]+= np.array(rewards)[1]
#        ep_scores += np.array(rewards)                                # update the score
#        agent.step(states, actions, rewards, next_states, dones, False)
        print("Rewards:",rewards)
        print("ep_scores:",ep_scores  )
        #print("MAX ep_scores:",np.maximum(np.array(ep_scores)))
        states = next_states                             # roll over the state to next time step
        if dones:                                       # exit loop if episode finished
            break
    scores=np.amax(ep_scores)    
    print("Episode Score: {}".format(scores))
print("Ended")


Rewards: [0.10000000149011612, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0. ]
Episode Score: 0.10000000149011

Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.0, 0.0]
ep_scores: [0.1 0.1]
Episode Score: 0.10000000149011612
Rewards: [0.

Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.0, 0.0]
ep_scores: [0.2 0.2]
Episode Score: 0.20000000298023224
Rewards: [0.10000000149011612, 0.0]
ep_scores: [0.3 0.2]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.2]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.2]
Episode Score: 0.30000000447034

Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, -0.009999999776482582]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.30000000447034836
Rewards: [0.0, 0.0]
ep_scores: [0.3 0.3]
Episode Score: 0.300000004470

Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.40000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.40000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.40000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.40000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.10000000149011612]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewards: [0.0, 0.0]
ep_scores: [0.50000001 0.50000001]
Episode Score: 0.5000000074505806
Rewar

...

Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.60000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.60000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.60000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.60000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.10000000149011612]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0.0]
ep_scores: [0.80000001 0.70000001]
Episode Score: 0.800000011920929
Rewards: [0.0, 0

Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.80000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
e

Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.0, 0.0]
ep_scores: [1.00000001 0.90000001]
Episode Score: 1.0000000149011612
Rewards: [0.10000000149011612, 0.0]
ep_scores: [1.10000002 0.90000001]
Episode Score: 1.1000000163912773
Rewards: [0.0, 0.0]
ep_scores: [1.10000002 0.90000001]
Episode Score: 1.1000000163912773
Rewar

Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
ep_scores: [1.20000002 0.90000001]
Episode Score: 1.2000000178813934
Rewards: [0.0, 0.0]
e

Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.0, 0.0]
ep_scores: [1.30000002 1.00000001]
Episode Score: 1.3000000193715096
Rewards: [0.10000000149011612, 0.0]
ep_scores: [1.40000002 1.00000001]
Episode Score: 1.4000000208616257
Rewards: [0.0, 0.0]
ep_scores: [1.40000002 1.00000001]
Episode Score: 1.4000000208616257
Rewards: [0.0, 0.0]
ep_scores: [1.40000002 1.00000001]
Episode Score: 1.4000000208616257
Rewards: [0.0, 0.0]
ep_scores: [1.40000002 1.00000001]
Episode Score: 1.4000000208616257
Rewar

### 6b. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
print('Random games ended')

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 